# Polygonization

In this workshop we will learn how to convert our topological model of the building into a geometric model.

In [44]:
import topogenesis as tg
import numpy as np 
import pyvista as pv
import os
import copy

In [2]:
two = 2 ** np.arange(8)
print(two)

[  1   2   4   8  16  32  64 128]


In [4]:
l_bis = []
for i in range(2**8):
    bi = np.array(list(np.binary_repr(i, width=8))).astype(int).reshape((2,2,2))
    l_bi = tg.to_lattice(bi, [0,0,0])
    l_bis.append(l_bi)

In [49]:
#  creating stencils
so_base = tg.create_stencil("von_neumann", 0, 1)
so_base.function = tg.sfunc.sum
sx_base = copy.deepcopy(so_base)
sx_base.set_index((0,0,0), 0)
sx_base.set_index((1,0,0), 1)

# setting separate direction
corner_state = dict(
    OO = so_base,
    XP = sx_base,
    XN = np.flip(sx_base, 0),
    YP = np.transpose(sx_base, (1,0,2)),
    YN = np.transpose(np.flip(sx_base, 0), (1,0,2)),
    ZP = np.transpose(sx_base, (2,1,0)),
    ZN = np.transpose(np.flip(sx_base, 0), (2,1,0))
)
# setting paired directions
corner_state["XX"] = corner_state["XN"] + corner_state["XP"]
corner_state["YY"] = corner_state["YN"] + corner_state["YP"]
corner_state["ZZ"] = corner_state["ZN"] + corner_state["ZP"]

# dummy symmetry string 
sym_str = [["OO"], ["XX", "YY", "ZZ"]]

stencils = []
for sym in sym_str:
    sym_stencil = copy.deepcopy(corner_state[sym[0]])
    for key in sym[1:]:
        sym_stencil += corner_state[key]
    stencils.append(sym_stencil)
    
print(stencils)
# print(corner_state["XX"])
# neighbour stencil
s_xyz = tg.create_stencil("von_neumann", 1, 1)
s_xyz.set_index((0,0,0), 0)
s_xyz.function = tg.sfunc.sum

# center stencil
s_o = tg.create_stencil("von_neumann", 0, 1)
s_o.function = tg.sfunc.sum

[stencil([[[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],

         [[0, 0, 0],
          [0, 1, 0],
          [0, 0, 0]],

         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]]]), stencil([[[0, 0, 0],
          [0, 1, 0],
          [0, 0, 0]],

         [[0, 1, 0],
          [1, 0, 1],
          [0, 1, 0]],

         [[0, 0, 0],
          [0, 1, 0],
          [0, 0, 0]]])]


In [25]:
indices_3d = np.array(np.unravel_index(np.arange(8), (2,2,2))).T

c = 0
results = []
for l in l_bis:
    # retrieve corner values for each stencil (flattened)
    result_o = l.apply_stencil(s_o).flatten()
    result_xyz = l.apply_stencil(s_xyz).flatten()

    # concatenate results into columns
    result = np.c_[result_o, result_xyz]

    results.append(result)

# stack results vertically
res_stacked = np.vstack(results)

# find the uniqe arangements of corners
uniq_corner_arang = np.unique(res_stacked, axis = 0)

In [27]:
# generate mini-lattices to represent the geometrical equivalent of unique corner arrangments
base_zero = np.zeros((2,2,2), dtype=np.int8)

corner_lattices = []
# for each unique arangement, create a representation
for uca in uniq_corner_arang:
    corner_arang = np.copy(base_zero)

    # set the corner
    corner_arang[0,0,0] = uca[0] == 1

    # set sides
    corner_arang[1,0,0] = uca[1] > 0
    corner_arang[0,1,0] = uca[1] > 1
    corner_arang[0,0,1] = uca[1] > 2

    # convert to lattice
    corner_lat = tg.to_lattice(corner_arang, np.zeros(3))
    corner_lattices.append(corner_lat)
    

In [31]:
p = pv.Plotter(notebook=True)

base_lattice = corner_lattices[0]

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# # adding the avilability lattice
# init_avail_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa")

def create_mesh(value):
    f = int(value)
    lattice = corner_lattices[f]

    # Add the data values to the cell data
    grid.cell_arrays["filled"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([.9, 1.1])
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)

    return

p.add_slider_widget(create_mesh, [0, len(corner_lattices)-1], title='Arrangements', value=0, event_type="always", style="classic", pointa=(0.1, 0.1), pointb=(0.9, 0.1))
p.show(use_ipyvtk=True)

  File "<ipython-input-31-88cad96a8dd7>", line 32, in create_mesh
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)
  File "/Users/shervinazadi/opt/miniconda3/envs/sc_workshops/lib/python3.8/site-packages/pyvista/plotting/plotting.py", line 1475, in add_mesh
    raise ValueError('Empty meshes cannot be plotted. Input mesh has zero points.')
ValueError: Empty meshes cannot be plotted. Input mesh has zero points.


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(4.363703305156274, 4.363703305156274, 4.363703305156274),
 (0.5, 0.5, 0.5),
 (0.0, 0.0, 1.0)]

### Credits

In [ ]:
__aut__author__ = "Shervin Azadi"
__license__ = "MIT"
__version__ = "1.0"
__url__ = "https://github.com/shervinazadi/spatial_computing_workshops"
__summary__ = "Spatial Computing Design Studio Workshop on Polygonization"